The principal objective of this code it's comparing the time of programing between use a regular query to a pyspark query. The table is locate in a remote database and it has more than 10 million registers. As i'm said, the whole objective its only comparing the mtime of programing and don't know about the results of the table it self.

All the codes were running in google colab.

In [2]:
import psycopg2 as pg
import pandas as pd
import time

def get_data(host, database, user, password, query):

  # get the start time
  st = time.time()
  try:
      connection = pg.connect(host = host, database = database, user = user, password = password)
      #Applying the query
      df = pd.read_sql(query, connection)  

  except (Exception, pg.Error) as error:
      print("Error while fetching data from PostgreSQL", error)

  finally:
      # closing database connection.
      if connection:
          connection.close()
          print("PostgreSQL connection is closed")

  # get the end time
  et = time.time()

  # get the execution time
  elapsed_time = et - st
  if(elapsed_time > 60): 
    elapsed_time = elapsed_time/60
    message = 'minute(s)'
  else:
    message = 'second(s)'

  print('#######################################################')
  print('Execution time:', round(elapsed_time, 2), message)
  print('#######################################################')

  return df


In [3]:
# Connecting to a simple database using the function get_data, all the credentials is confidential
host = 'host'
database = 'database'
user = 'user'
password = 'password'
query = 'SELECT * FROM "table"'

df = get_data(host, database, user, password, query)

PostgreSQL connection is closed
#######################################################
Execution time: 3.27 minute(s)
#######################################################


In [ ]:
# Showing the number of rows
len(df.index)

11054769

Construing the pyspark structure

In [21]:
# installing the dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

#Intalling postgres drive for spark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-30 01:54:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.08MB/s    in 0.2s    

2022-09-30 01:54:45 (5.08 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [22]:
# Configure environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Make pyspark "importable"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [32]:
import time
# get the start time
st = time.time()

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://host:port/database") \
    .option("dbtable", "dbtable") \
    .option("user", "user") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

  # get the end time
et = time.time()

  # get the execution time
elapsed_time = et - st
if(elapsed_time > 60): 
    elapsed_time = elapsed_time/60
    message = 'minute(s)'
else:
    message = 'second(s)'

print('#######################################################')
print('Execution time:', round(elapsed_time, 2), message)
print('#######################################################')    

#######################################################
Execution time: 0.52 second(s)
#######################################################


In [ ]:
type(df)